In [2]:
import numpy as np
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPModel, CyLPArray
import pandas as pd
import time
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [3]:
def var_matrix(n): # cette matrice calcule la différence de production d'énergie entre deux périodes consécutives
    M = np.zeros((n, n+1))
    for i in range(n):
        M[i,i] = -1
        M[i,i+1] = 1
    return np.matrix(M)


def matrix_sum_periods(N, T): # somme les productions d'énergie en périodes de T heures
    M = np.zeros((N//T, N))
    for i in range(N//T):
        M[i, i*T:(i+1)*T] = ([1] * T)
    
    return np.matrix(M)


def sum_by_col(matrix,N):
    sum_c = np.sum(np.array(matrix),axis = 0)
    sum_co = sum_c[:N]
    sum_co = CyLPArray(sum_co)
    return sum_co
    

def sum_rendement(rend, nb_site): # somme les rendements de chaque site sur toutes les heures / fonction objectif
    sum_rend = np.sum(rend, axis=1)
    c = np.ndarray((nb_site,))
    for i in range(nb_site):
        c[i] = sum_rend[i,0]
    c = CyLPArray(c)
    return c

def matrix_rendement(rend_off, rend_on, sites): # sélectionne les bonnes lignes des matrices de rendement
    lignes = len(rend_off)
    colonnes = len(rend_off[0])
    
    M = np.empty((lignes, colonnes))
    for i, site in sites.iterrows():
        if site['capacite offshore'] == 'Oui':
            M[i] = rend_off[i]
        else:
            M[i] = rend_on[i]
    
    return np.matrix(M)

def vecteur_OffouOn(sites,nb_sites,N):
    M1 = np.zeros(nb_sites)
    M2 = np.zeros(nb_sites)
    for i,site in sites.iterrows():
        if site['capacite offshore'] == 'Oui':
            M1[i] = cout_instal_offshore*(N/8760)
        else :
            M2[i] = cout_instal_onshore*(N/8760)
    return CyLPArray(M1 + M2)

In [4]:
N = 24*7
nb_site = 642


sites = pd.read_csv('data/Sites.csv')
sites = sites.sort_values('index site')
sites = sites.reset_index()
capacites = sites['capacites'].copy()
capacites = CyLPArray(capacites) # shape: (642,)


off_cap = sites['capacite offshore']
off_cap = off_cap.replace('Oui', 1)
off_cap = off_cap.replace('Non', 0)
off_cap = CyLPArray(off_cap)


rend_off_entire = np.genfromtxt('data/Rendements_offshore.csv', delimiter=',')
rend_on_entire = np.genfromtxt('data/Rendements_onshore.csv', delimiter=',')

#---------------------------------


# Consommation électrique (demande) par pays et par heure (matrice : ligne = pays, colonne = heure)
consommations = np.genfromtxt('data/Consommations.csv', delimiter=',')[:,:N] # shape: (15, N)

# Couts d'installation amortis des éoliennes
cout_instal_onshore  = 168903 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)
cout_instal_offshore = 300336 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)
 
# Couts d'installation et de fonctionnement des centrales au gaz
cout_amorti_gaz = 94956 # Coût *amorti sur un an* pour l'installation d'un MW de centrale au gaz (euros/MW/an)
cout_fonct_gaz  = 65    # Coût de fonctionnement pour la production d'un MWh par centrale au gaz (euros/MWh)

## Données hydroélectriques 

# Stockage disponible par pays (en MWh)
stockage_hydro = np.array([0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6])

# Puissances maximales de turbinage et de pompage disponibles par pays (en MW)
p_turbinage = np.array([8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101])
p_pompage = np.array([5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ])

eta_turbinage = 0.75 # Rendement pour le turbinage (sans unité)

# Apports naturels au stockagep par pays et par heure
apports = np.genfromtxt('data/Apports-hydro.csv', delimiter=',')[:,:N] # shape: (15, N)

M = stockage_hydro.sum()
P_p = p_pompage.sum()
P_t = p_turbinage.sum()


T = 3

In [78]:
#  DEBUGGING

rend_off = rend_off_entire[:,:N]
rend_on = rend_on_entire[:,:N]


rend = matrix_rendement(rend_off, rend_on, sites) # shape: (642, N)
I = np.matrix(np.eye(nb_site)) # shape: (642, 642)
A = CyLPArray(matrix_sum_periods(N, T)@sum_by_col(apports,N))[0] # shape: (N/T,)
D = CyLPArray(matrix_sum_periods(N, T)@sum_by_col(consommations, N))[0] # shape: (N/T,)
R = np.matrix(matrix_sum_periods(N,T)@rend.T) # shape: (N/T, 642)

print(I.shape)
print(A.shape)
print(D.shape)
print(R.shape)

print(M)
print(P_p)
print(P_t)

(642, 642)
(56,)
(56,)
(56, 642)
169958000.0
35546
158347


In [151]:
rend_off = rend_off_entire[:,:N]
rend_on = rend_on_entire[:,:N]


rend = matrix_rendement(rend_off, rend_on, sites) # shape: (642, N)
I = np.matrix(np.eye(nb_site)) # shape: (642, 642)
A = CyLPArray(matrix_sum_periods(N, T)@sum_by_col(apports,N))[0] # shape: (N/T,)
D = CyLPArray(matrix_sum_periods(N, T)@sum_by_col(consommations, N))[0] # shape: (N/T,)
R = np.matrix(matrix_sum_periods(N,T)@rend.T) # shape: (N/T, 642)



model = CyLPModel()
X   = model.addVariable('X', nb_site)
B   = model.addVariable('B',N//T +1 )
H_m = model.addVariable('H_m',N//T )
H_d = model.addVariable('H_d',N//T)
E   = model.addVariable('E',N//T )

factor = np.matrix(np.eye(N//T)/eta_turbinage)

model.addConstraint(E + H_d >= D)
model.addConstraint(R*X - E - H_m == 0)
model.addConstraint(var_matrix(N//T)*B - H_m + np.matrix(np.eye(N//T)/eta_turbinage)*H_d == A)

model.addConstraint(0 <= X <= capacites, 'capacitee')
model.addConstraint(0 <= B <= M)
model.addConstraint(0 <= H_m <= T*P_p)
model.addConstraint(0 <= H_d <= T*P_t)

model.addConstraint(B[0] == M/2)
model.addConstraint(B[-1] == M/2)
    

C = vecteur_OffouOn(sites, nb_site, N)

model.objective =  C * X

print('création du simplex...', end='')
s = CyClpSimplex(model)
print('done')


print(s.dual())
print('solution:', s.objectiveValue)

solB  = s.primalVariableSolution['B']
solX  = s.primalVariableSolution['X']
solHm = s.primalVariableSolution['H_m']
solHd = s.primalVariableSolution['H_d']
solE  = s.primalVariableSolution['E']

création du simplex...done
optimal
solution: 2282760745.888384


In [193]:
# vérification des contraintes

#1

test_capacites = True
for i in range(len(capacites)):
    if solX[i] - capacites[i] > 1e-5:
        test_capacites = False
        print('ERREUR: indice {}, {} > {}'.format(i, solX[i], capacites[i]))
if test_capacites:
    print('test capacites: OK')
    
# 2

solBVar = np.array(var_matrix(N//T)@solB)[0]
test_diffB = True
for i in range(len(solBVar)):
    if abs(solBVar[i] - (A[i] + solHm[i] - solHd[i]/eta_turbinage)) > 1e-5:
        test_diffB = False
        print('ERREUR: {} != {} + {} - {}/eta'.format(solBVar[i], A[i], solHm[i], solHd[i]))
if test_diffB:
    print('test différence barrage: OK')

# 3

test_demande = True
for i in range(len(solE)):
    if D[i] - solE[i] - solHd[i] > 1e-5:
        test_demande = False
        print('ERREUR: indice {}, demande: {}, dispo: {} + {} = {}'.format(i, D[i], solE[i], solHd[i], solE[i] + solHd[i]))
if test_demande:
    print('test demande: OK')

# 4

test_production = True
for i in range(len(solE)):
    if abs( np.array(R@solX)[0][i] - solE[i] - solHm[i] ) > 1e-5:
        test_production = False
        print('ERREUR')
if test_production:
    print('test production: OK')

test capacites: OK
test différence barrage: OK
test demande: OK
test production: OK


In [1]:
# quelques plots pour tester

plt.plot(solB)
plt.plot([M/2]*len(solB))
plt.title('état du barrage')
plt.legend(['niveau du barrage', 'moitié de la capacité'])
plt.show()

plt.plot(D)
plt.plot(solE)
plt.plot(solHd)
plt.plot(solE+solHd)
plt.title('arrivée d\'énergie au village')
plt.legend(['demande', 'énergie éolienne', 'énergie hydroélectrique', 'somme des 2 inputs'])
plt.show()

plt.plot(solHd/eta_turbinage)
plt.plot(solHm)
plt.plot(A)
plt.plot([106638] * len(A), '--r', linewidth=1)
plt.plot([633388] * len(A), '--r', linewidth=1)
plt.title('pompage, turbinage et apport en eau')
plt.legend(['turbinage', 'pompage', 'apport en eau'])
plt.show()

NameError: name 'plt' is not defined

In [196]:
tot = A.sum() + solHm.sum() - (solHd/eta_turbinage).sum()
print(tot)
print(T*P_p, T*P_t/eta_turbinage)
print(P_p, P_t)

-5.587935447692871e-09
106638 633388.0
35546 158347
